### OPEN API 를 이용한 챗봇 시스템 

In [1]:
!pip show openapi
# !pip install openai

Name: openapi
Version: 2.0.0
Summary: Python OpenAPI 2.0 (Swagger) object model
Home-page: https://github.com/globality-corp/openapi
Author: Globality Engineering
Author-email: engineering@globality.com
License: 
Location: /opt/anaconda3/lib/python3.11/site-packages
Requires: inflection, jsonschema
Required-by: 


---
### OPENAI API 호출 예제 

In [2]:
from openai import OpenAI
from pprint import pprint
import os
api_key=os.environ.get("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

client = OpenAI(api_key=api_key)
model = 'gpt-3.5-turbo-1106'

messages = [
    {'role':'system','content': 'You are a helphful assistant.'},
    {'role':'user','content': '군자역 혼잡도 알려줘?'}
]
response = client.chat.completions.create(model=model, messages=messages).model_dump()
pprint(response)

{'choices': [{'finish_reason': 'stop',
              'index': 0,
              'logprobs': None,
              'message': {'content': '죄송합니다, 현재 군자역의 혼잡도 정보는 알려드릴 수 없습니다. 군자역 '
                                     '근처의 역 관리소나 공식 웹사이트에서 실시간 정보를 확인하시는 것을 '
                                     '권장드립니다.',
                          'function_call': None,
                          'role': 'assistant',
                          'tool_calls': None}}],
 'created': 1720085417,
 'id': 'chatcmpl-9hD575rBfhSnyAiaTc7r7jAmM2lbq',
 'model': 'gpt-3.5-turbo-1106',
 'object': 'chat.completion',
 'service_tier': None,
 'system_fingerprint': 'fp_44132a4de3',
 'usage': {'completion_tokens': 82, 'prompt_tokens': 36, 'total_tokens': 118}}


> gpt 는 json type 으로 답을 넘겨준다. 

---
### 

 * [model dump ]
    : 메서드 체이닝 -> 메서드를 연쇄적으로 호출하는 방식 

---
### ai chatbot 을 만들기 위해서는 프롬프트 엔지니어링이 필요하다.

In [3]:
prompt = """
내일 다음 주제로 미래 교육 포럼에서 발표할 발제문을 작성해야합니다.
발제문의 주요 내용을 불릿 기호로 간략히 정리해 주세요.
주제:{agenda}


""".format(agenda="인공지능 시대의 교육의 역활과 의미")
model = 'gpt-3.5-turbo-1106'
response = client.chat.completions.create(
    model= model,
    messages=[{'role':'user','content':prompt}]
).to_dict
pprint(response)



<bound method BaseModel.to_dict of ChatCompletion(id='chatcmpl-9hD58Rx4Q2trfXlep9qvKGWw8iftV', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='- 인공지능 기술의 발전과 교육의 관련성\n- 인공지능이 교육에 미치는 영향과 변화\n- 학교 교육과 인공지능의 융합 가능성\n- 인공지능을 효과적으로 활용한 교육 방법과 도구\n- 교사와 학생의 역할 변화와 대비책\n- 인공지능과 교육의 윤리적 문제들에 대한 대처 전략', role='assistant', function_call=None, tool_calls=None))], created=1720085418, model='gpt-3.5-turbo-1106', object='chat.completion', service_tier=None, system_fingerprint='fp_44132a4de3', usage=CompletionUsage(completion_tokens=152, prompt_tokens=97, total_tokens=249))>


In [4]:
from pprint import pprint

prompt = """
내일 다음 주제로 미래 교육 포럼에서 발표할 발제문을 작성해야합니다.
발제문의 주요 내용을 불릿 기호로 간략히 정리해 주세요.
주제:{agenda}
""".format(agenda="인공지능 시대의 교육의 역할과 의미")

model = 'gpt-3.5-turbo-1106'
response = client.chat.completions.create(
    model=model,
    messages=[{'role': 'user', 'content': prompt}]
)

pprint(response.choices[0].message.content)


('- 인공지능이 발전함에 따라 교육의 역할이 어떻게 변화하는가?\n'
 '- 인공지능이 교육에 미치는 영향은 무엇인가?\n'
 '- 학생들을 준비하고 미래를 대비하는 데 교육이 어떤 역할을 해야 하는가?\n'
 '- 인공지능과 교육의 상호작용이 미래 사회에 어떠한 의미를 갖는가?')


---
### Chater 5 프롬프트 엔지니어링의 핵심 기법 
- few shot
- Cot
-SC
- ToT
- ReAct
-RAG


In [4]:
def GPT_prompt(model = 2, template_go = False , prompt=""):
    from pprint import pprint
    import json
    from openai import OpenAI
    from pprint import pprint
    import os
    api_key=os.environ.get("OPENAI_API_KEY")
    client = OpenAI(api_key=api_key)
    
    if template_go :
        
        template = """
        당신은 번역함수이며, 반환값은 반드시 JSON 데이터 여야 합니다. 
        STEP 별로 작업을 수행하면서 그 결과를 아래의 출력 결과 JSON 포맷에 작성하세요.
        STEP-1. 아래 세 개의 백틱으로 구분된 텍스트를 원문 그대로 읽어 올것
        STEP-2. 입력받은 텍스트가 긍정적이라면 True를 표기할것
        STEP-3. 다음의 말투로 번역할 것: ["지구의 나이는 45억살이야.","세종 대왕은 조선의 위대한 국왕이야."]
        ```{text}```
        ---
        출력결과 : {{"STEP-1":<입력텍스트>, "STEP-2": <true/false>, "STEP-3":<번역결과>}}
        """
        text = "William Shakespeare was an English playwrit, poet and actor.  He is widely regarded as the greatest writer in the English language and the world's pre-eminent dramatist."
        
        template = template.format(text=text)
        context = [{"role": "user","content": template}]
        model_1 = 'gpt-3.5-turbo-1106'
        model_2 = 'gpt-4-0613'
        response = client.chat.completions.create(
            model=model_2 if model == 2 else model_1,
            messages=context,
            temperature=0,
            top_p=0,
            seed=1234
        ).model_dump()
        pprint(json.loads(response['choices'][0]['message']['content']))
        return json.loads(response['choices'][0]['message']['content'])
    else :
        template = prompt
        context = [{"role": "user","content": template}]

        model_1 = 'gpt-3.5-turbo-1106'
        model_2 = 'gpt-4-0613'
        response = client.chat.completions.create(
            model=model_2 if model == 2 else model_1,
            messages=context,
            temperature=0,
            top_p=0,
            seed=1234
        ).model_dump()
        print(response['choices'][0]['message']['content'])
    
    
    # pprint(response.choices[0].message.content)
GPT_prompt(prompt="너 바보야?")

저는 인공지능으로, 감정이나 지식을 가지고 있지 않습니다. 그러므로 바보라는 개념이 적용되지 않습니다. 어떻게 도와드릴까요?


---
###  프롬프트 가이드라인
- 과업배경: 당신은 번역함수
- 원하는 결과 예시 
- 단계별 가이드 
- 제약사항 
- 수행결과 작성 형식 : 출력결과 : {{"STEP-1":<입력텍스트>, "STEP-2": <true/false>, "STEP-3":<번역결과>}}
- 적절한 구분자의 사용
- 구체적이면서 명확한 과업지시


---
#### 인컨텍스트 러닝이란?
- 학습햇던 내용을 바탕으로 답하는 것이 아니라, 런타임 시점에 제공되는 입력정보를 바탕으로 추로하는 능력 : 주어진 문맥안에서 그 의미를 배운다
- 언어모델 추론능력 향상 -> 프롬프트 엔지니어링의 출발 

- Few-shot Prompting 
    : 그 질문과 관련된 예시를 적절히 제공하면 언어 모델이 그 예시의 의미를 추론하여 답을 말하는원리

## Zero shot prompting 

In [7]:
## Zero shot prompting 
from Functions import NLP_Service as N
template = """
긍정 또는 부정으로 답변을 작성하세요.
Q: 매력적인 이성과 사랑에 빠졌어!
A: 
"""

N.GPT_prompt(prompt=template)

긍정


In [2]:
## Few shot prompting 
from Functions import NLP_Service as N
template = """
아래 예시를 참조해 마지막 답변을 긍정 또는 부정으로 작성하세요

```
Q: 난 오늘 기분이 나빠 
A: 긍정
```
Q: 드디서 사업에 성공했어
A: 부정
```
Q: 요즘 너무 행복해
A: 부정
```
Q: 슬픈일이 벌어졌어
A: 긍정
```
Q: 매력적인 이성과 사랑에 빠졌어!
A: < 정답을 작성하고 그렇게 답한 이유를 말하세요>

"""
N.GPT_prompt(prompt=template)


('A: 부정 \n'
 '\n'
 '이유: 이 질문의 답변은 주어진 예시의 패턴을 따르면 부정적인 답변이 되어야 합니다. 예시에서는 긍정적인 상황에는 부정적인 답변을, '
 '부정적인 상황에는 긍정적인 답변을 하고 있습니다. 따라서 "매력적인 이성과 사랑에 빠졌어!"는 긍정적인 상황이므로 부정적인 답변을 해야 '
 '합니다.')


---
### JSON 모드 대답 ( 2023.11.6 도입)

In [2]:
def GPT_prompt_json(model = 2, template_go = False , prompt="", json_mode=True):
    from pprint import pprint
    import json
    from openai import OpenAI
    from pprint import pprint
    import os
    api_key=os.environ.get("OPENAI_API_KEY")
    client = OpenAI(api_key=api_key)
    model_1 = 'gpt-3.5-turbo-1106'
    model_2 = 'gpt-4-0613'
    context = [{"role": "user","content": "한국 초대 대통령 이름, 출생일자, 사망일자를 json 으로 기술하세요"}]
    completion = client.chat.completions.create(
        model= model_1,
        messages = context,
        response_format ={"type":"json_object"}
    )
    print(json.loads(completion.choices[0].message.content))
    
    # pprint(response.choices[0].message.content)
GPT_prompt(prompt="너 바보야?")

{'이름': '이승만', '출생일자': '1875-11-10', '사망일자': '1965-08-19'}


---
### 챗봇 시스템 구현 
- 대화를 주고 받는 멀티턴 방식 
- gpt ->완성형, 대화형 언어모델이 있음. 
- gpt3.5 는 완성형 모델  
#### 챗봇 시스템 설계 
